<a href="https://colab.research.google.com/github/barbosacesar/Intelig-ncia-Computacional/blob/main/ProjetoDisciplina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --no-cache-dir numpy==1.26.4 pandas ipywidgets opencv-python-headless fer matplotlib moviepy

In [7]:
!pip install deepface

In [9]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [10]:
# Montar Google Drive para acessar seus arquivos
from google.colab import drive
drive.mount('/content/drive')

# Mudar diretório para sua pasta de trabalho
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [18]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import time
import cv2
import uuid
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import zipfile
import os
import tempfile
import numpy as np
from PIL import Image, UnidentifiedImageError
from deepface import DeepFace

EMOCOES_INFO = {
    'happy': ('Feliz', '😄', '#4CAF50'),
    'sad': ('Triste', '😢', '#1E88E5'),
    'angry': ('Bravo', '😠', '#E53935'),
    'surprise': ('Surpreso', '😲', '#FFEB3B'),
    'neutral': ('Neutro', '😐', '#9E9E9E'),
    'fear': ('Medo', '😨', '#8E24AA'),
    'disgust': ('Nojo', '🤢', '#6D4C41'),
}

def emocao_info(emocao):
    return EMOCOES_INFO.get(str(emocao).lower(), ('Desconhecida', '❓', '#000000'))

saida = widgets.Output()

def debug_file(filepath):
    """Função para diagnóstico de arquivos"""
    print(f"\nDebugging: {filepath}")
    print(f"Tamanho: {os.path.getsize(filepath)} bytes")
    with open(filepath, 'rb') as f:
        print(f"Primeiros bytes: {f.read(8)}")
    try:
        with Image.open(filepath) as img:
            print(f"Formato PIL: {img.format}")
    except Exception as e:
        print(f"Erro PIL: {str(e)}")

def is_valid_image_file(filepath):
    """Verifica se um arquivo é uma imagem válida"""
    try:
        if not os.path.exists(filepath):
            print(f"Arquivo não encontrado: {filepath}")
            return False

        if os.path.getsize(filepath) == 0:
            print(f"Arquivo vazio: {filepath}")
            return False

        # Verificação com Pillow
        try:
            with Image.open(filepath) as img:
                img.verify()

                # Verificação adicional para PNG
                if filepath.lower().endswith('.png'):
                    with open(filepath, 'rb') as f:
                        header = f.read(8)
                        if header != b'\x89PNG\r\n\x1a\n':
                            print(f"Assinatura PNG inválida em: {filepath}")
                            return False
            return True
        except Exception as e:
            print(f"Falha na verificação com Pillow: {filepath} - {str(e)}")
            return False

    except Exception as e:
        print(f"Erro na validação da imagem {filepath}: {str(e)}")
        return False

def has_face(image_path):
    """Verifica se a imagem contém rostos detectáveis"""
    try:
        if not is_valid_image_file(image_path):
            return False

        # Carrega a imagem convertendo via PIL para OpenCV
        with Image.open(image_path) as img:
            img = img.convert('RGB')
            img_array = np.array(img)
            img_cv2 = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        # Carrega o classificador Haar Cascade
        face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )

        if face_cascade.empty():
            print("Erro: Classificador Haar Cascade não carregado")
            return False

        # Detecta faces (CORREÇÃO APLICADA AQUI)
        gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)  # Fechando corretamente os parênteses
        )

        return len(faces) > 0

    except Exception as e:
        print(f"Erro na detecção de faces {image_path}: {str(e)}")
        return False

def process_image(filepath):
    """Processa uma imagem de forma robusta"""
    try:
        # Debug (opcional)
        # debug_file(filepath)

        if not is_valid_image_file(filepath):
            print(f"Arquivo de imagem inválido: {filepath}")
            return None

        # Carrega via PIL e converte para OpenCV
        with Image.open(filepath) as img:
            img = img.convert('RGB')
            img_array = np.array(img)
            img_cv2 = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)

        return img_cv2

    except Exception as e:
        print(f"Erro no processamento da imagem {filepath}: {str(e)}")
        return None

def get_facial_attributes(filepath, conf_threshold=0.3):
    """Analisa atributos faciais com tratamento robusto de erros"""
    try:
        img_cv2 = process_image(filepath)
        if img_cv2 is None:
            return []

        # Cria arquivo temporário JPEG para análise
        temp_jpg = os.path.join(tempfile.gettempdir(), f"temp_{uuid.uuid4().hex}.jpg")
        cv2.imwrite(temp_jpg, img_cv2)

        try:
            # Análise com DeepFace
            results = DeepFace.analyze(
                img_path=temp_jpg,
                actions=['emotion'],
                enforce_detection=True,
                detector_backend='opencv',
                silent=True
            )

            if not results:
                return []

            # Processa resultados
            if isinstance(results, list):
                results = results[0]

            dominant_emotion = results["dominant_emotion"]
            confidence = results["emotion"][dominant_emotion] / 100

            if confidence < conf_threshold:
                print(f"Confiança baixa em {os.path.basename(filepath)}: {confidence:.2f}")
                return []

            return [{
                "facial_expression": dominant_emotion,
                "confidence": confidence,
                "emotions": results["emotion"]
            }]

        finally:
            if os.path.exists(temp_jpg):
                os.remove(temp_jpg)

    except Exception as e:
        print(f"Erro na análise facial de {os.path.basename(filepath)}: {str(e)}")
        return []

def analisar_video(caminho_video, max_frames=10):
    contagem_emocoes = Counter()
    frame_count = 0

    try:
        cap = cv2.VideoCapture(caminho_video)
        if not cap.isOpened():
            print(f"Erro ao abrir vídeo {caminho_video}")
            return contagem_emocoes

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = max(1, total_frames // max_frames) if total_frames > max_frames else 1

        while cap.isOpened() and frame_count < max_frames:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count * frame_interval)

            temp_file = os.path.join(tempfile.gettempdir(), f"frame_{uuid.uuid4().hex}.jpg")
            cv2.imwrite(temp_file, frame)

            try:
                resultado = get_facial_attributes(temp_file)
                if resultado:
                    emocao = resultado[0]["facial_expression"]
                    contagem_emocoes[emocao] += 1
            finally:
                if os.path.exists(temp_file):
                    os.remove(temp_file)

            frame_count += 1

    except Exception as e:
        print(f"Erro ao processar vídeo {caminho_video}: {str(e)}")
    finally:
        if 'cap' in locals() and cap.isOpened():
            cap.release()

    return contagem_emocoes

def on_upload_change(change):
    if not uploader.value:
        return

    arquivos = list(uploader.value.values())
    contagem_emocoes = Counter()
    total_start = time.time()

    with saida:
        clear_output(wait=True)
        print(f"{len(arquivos)} arquivo(s) enviado(s). Iniciando análise...")

    temp_files = []

    try:
        for arquivo in arquivos:
            filename = arquivo['metadata']['name']
            content = arquivo['content']

            # Cria arquivo temporário com extensão original
            temp_ext = os.path.splitext(filename)[1]
            temp_file = os.path.join(tempfile.gettempdir(), f"temp_{uuid.uuid4().hex}{temp_ext}")

            try:
                with open(temp_file, 'wb') as f:
                    f.write(content)
                temp_files.append(temp_file)

                if filename.lower().endswith('.zip'):
                    pasta_temp = os.path.join(tempfile.gettempdir(), f"extracted_{uuid.uuid4().hex}")
                    os.makedirs(pasta_temp, exist_ok=True)
                    temp_files.append(pasta_temp)

                    try:
                        with zipfile.ZipFile(temp_file, 'r') as zip_ref:
                            zip_ref.extractall(pasta_temp)
                    except Exception as e:
                        print(f"Erro ao descompactar {filename}: {str(e)}")
                        continue

                    imagens = []
                    for ext in ('*.jpg', '*.jpeg', '*.png'):
                        imagens.extend(Path(pasta_temp).rglob(ext))

                    for img_path in imagens:
                        try:
                            resultado = get_facial_attributes(str(img_path))
                            if resultado:
                                emocao = resultado[0]["facial_expression"]
                                contagem_emocoes[emocao] += 1
                        except Exception as e:
                            print(f"Erro ao analisar {img_path.name}: {str(e)}")

                elif filename.lower().endswith(('.avi', '.mp4', '.mov')):
                    try:
                        contagem_video = analisar_video(temp_file, max_frames=10)
                        for emocao, count in contagem_video.items():
                            contagem_emocoes[emocao] += count
                    except Exception as e:
                        print(f"Erro ao analisar vídeo {filename}: {str(e)}")

                elif filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    try:
                        resultado = get_facial_attributes(temp_file)
                        if resultado:
                            emocao = resultado[0]["facial_expression"]
                            contagem_emocoes[emocao] += 1
                    except Exception as e:
                        print(f"Erro ao analisar {filename}: {str(e)}")

                else:
                    print(f"{filename} ignorado (formato inválido).")

            except Exception as e:
                print(f"Erro ao processar {filename}: {str(e)}")

    finally:
        # Limpeza de arquivos temporários
        for filepath in temp_files:
            try:
                if os.path.isdir(filepath):
                    for root, dirs, files in os.walk(filepath, topdown=False):
                        for name in files:
                            os.remove(os.path.join(root, name))
                        for name in dirs:
                            os.rmdir(os.path.join(root, name))
                    os.rmdir(filepath)
                elif os.path.exists(filepath):
                    os.remove(filepath)
            except Exception as e:
                print(f"Erro ao limpar arquivo temporário {filepath}: {str(e)}")

    total_end = time.time()

    with saida:
        clear_output(wait=True)
        total = sum(contagem_emocoes.values())
        if total == 0:
            print("Nenhuma emoção detectada nas imagens ou vídeos.")
            return

        print(f"Total de emoções detectadas: {total}")
        print(f"Tempo total: {total_end - total_start:.2f} segundos\n")

        print("Contagem por emoção:")
        for emocao, count in contagem_emocoes.items():
            pt, emoji, _ = emocao_info(emocao)
            print(f"{emoji} {pt:<10}: {count} ({count/total:.1%})")

        try:
            labels = []
            values = []
            cores = []

            for emocao, count in contagem_emocoes.items():
                pt, emoji, cor = emocao_info(emocao)
                labels.append(f"{emoji} {pt}")
                values.append(count)
                cores.append(cor)

            plt.figure(figsize=(10, 5))
            plt.bar(labels, values, color=cores)
            plt.title("Distribuição das Emoções Detectadas")
            plt.xlabel("Emoções")
            plt.ylabel("Frequência")
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print("Erro ao gerar gráfico:", str(e))

    uploader.value.clear()

def mostrar_menu():
    global uploader
    uploader = widgets.FileUpload(
        accept='.png,.jpg,.jpeg,.zip,.avi,.mp4,.mov',
        multiple=True,
        description='Enviar Arquivo',
        style={'button_color': '#2196F3', 'font_weight': 'bold', 'font_size': '16px'}
    )
    uploader.observe(on_upload_change, names='value')

    botao_webcam = widgets.Button(
        description="Capturar Webcam",
        button_style='info',
        layout=widgets.Layout(width='160px'),
        style={'font_weight': 'bold', 'font_size': '16px'}
    )
    botao_webcam.on_click(lambda x: print("Função de captura via webcam ainda não implementada"))

    botoes = widgets.HBox([uploader, botao_webcam],
                         layout=widgets.Layout(justify_content='center', gap='20px', padding='10px'))

    display(HTML("<h2 style='text-align:center;'>Análise de Emoções Faciais</h2>"))
    display(HTML("<p style='text-align:center; font-size:16px;'>Envie uma imagem, vídeo ou capture da webcam</p>"))
    display(botoes)
    display(saida)

mostrar_menu()

Output()

Erro na análise facial de temp_3c5f73f6a97845a6ba3414c4d23f0181.png: Unable to synchronously open file (file signature not found)
